In [ ]:
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'deeplabv3_resnet101', pretrained=True)
model.eval()

In [ ]:
from PIL import Image
import numpy as np
from torchvision import transforms
# img = Image.open("/home/thyde/avatar-stuff/frames/s01e01/frame2990.jpg")


# def img_transform(img):
#     def to_rgb(pil):
#         return pil.convert('RGB')
#     image_transform = transforms.Compose([
#         transforms.Resize(500),
#         transforms.CenterCrop(500),
#         to_rgb,
#         transforms.ToTensor(),
#         transforms.Normalize(
#             mean=[0.485, 0.456, 0.406],
#             std=[0.229, 0.224, 0.225]),
#     ])
#     return image_transform(img)

# input_tensor = img_transform(img)
# input_batch = input_tensor.unsqueeze(0)

In [ ]:
# if torch.cuda.is_available():
#     input_batch = input_batch.to('cuda')
#     model.to('cuda')

# with torch.no_grad():
#     output = model(input_batch)['out'][0]
# output_predictions = output.argmax(0)

In [ ]:
# def get_segmaps(input_batch, model, reduce_max=False, to_numpy=False):
#     out = model(input_batch)
#     segmaps = out['out']
#     if reduce_max:
#         segmaps = segmaps.argmax(1)
#     if to_numpy:
#         segmaps = segmaps.data.numpy()
#     return segmaps

# segmap = get_segmaps(input_batch, model, reduce_max=True)


In [ ]:
# 0=background, 1=aeroplane, 2=bicycle, 3=bird, 4=boat, 5=bottle
# 6=bus, 7=car, 8=cat, 9=chair, 10=cow
# 11=dining table, 12=dog, 13=horse, 14=motorbike, 15=person
# 16=potted plant, 17=sheep, 18=sofa, 19=train, 20=tv/monitor

In [ ]:
import pickle
with open('/home/thyde/avatar-stuff/frames/person_frames.pickle','rb') as f:
    person_frames = pickle.load(f)

In [ ]:
person_frames[-1]

In [ ]:
import os
img_paths = os.listdir('/home/thyde/avatar-stuff/frames/s01e09')

In [ ]:
import torchvision
from torch.utils.data import DataLoader
image_transform = transforms.Compose([
    transforms.Resize(600),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]),
])


class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path


dataset = ImageFolderWithPaths('/home/thyde/avatar-stuff/frames/s01e09', transform=image_transform)
dataloader = DataLoader(dataset, 
                       shuffle=True, 
                       batch_size=12,
                      num_workers=8)

In [ ]:
import pickle
import frogress
# person_frames = list()
model.to('cuda')
for batch in frogress.bar(dataloader, steps=len(dataloader)):
    input_batch = batch[0]
    input_batch = input_batch.to('cuda')
    with torch.no_grad():
        segmaps = model(input_batch)['out'].argmax(1)
#     import pdb;pdb.set_trace()
    for idx, segmap in enumerate(segmaps):
        if 15 in segmap:
            if batch[2][idx] not in person_frames:
                person_frames.append(batch[2][idx])

with open('/home/thyde/avatar-stuff/frames/person_frames.pickle', 'wb') as fp:
    pickle.dump(person_frames, fp)


In [ ]:
len(person_frames)